In [29]:
#*** Craw data from cafef.vn***
import requests
from bs4 import BeautifulSoup, SoupStrainer
from datetime import datetime
import csv
from tqdm import tqdm

url = "https://cafef.vn/timelinelist/18831/{}.chn"
url_base="https://cafef.vn"
date_format = "%Y-%m-%dT%H:%M:%S"

with open('NewsPrediction.csv', mode='w', encoding='utf-8', newline='') as csv_file:
    fieldnames = ['date', 'content']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()

    with requests.Session() as session:
        for page in tqdm(range(0, 200)):
            response = session.get(url.format(page))
            soup = BeautifulSoup(response.content, "html.parser", parse_only=SoupStrainer("div", {"class": "tlitem box-category-item"}))
            news_list = soup.find_all("div", {"class": "tlitem box-category-item"})
            if not news_list:
                break
            for news in news_list:
              date = datetime.strptime(news.find("span", {"class": "time time-ago"}).text.strip(), date_format)
              article_url = url_base + news.a['href']
              article_response = requests.get(article_url)
              article_soup = BeautifulSoup(article_response.text, 'html.parser')
              content = ''
              paragraphs = article_soup.find_all('p')
              for paragraph in paragraphs:
                    content += paragraph.text.strip() + ' '
              writer.writerow({'date': date, 'content': content})
            # Dừng một chút để tránh bị chặn IP
            #time.sleep(2)


100%|██████████| 200/200 [21:18<00:00,  6.39s/it]


In [ ]:
#*** Install library ***
!pip install ftfy
!pip install underthesea
!pip install nltk
!pip install pyvi

In [4]:
#*** Gom các file dữ liệu tin tức về 1 file***
import pandas as pd

# Đọc 2 file CSV
df1 = pd.read_csv('1000.csv')
df2 = pd.read_csv('2000.csv')
df3 = pd.read_csv('3000.csv')
df4 = pd.read_csv('4000.csv')
df5 = pd.read_csv('6000.csv')
df6 = pd.read_csv('8000.csv')
df7 = pd.read_csv('10000.csv')
df8 = pd.read_csv('12000.csv')
df9 = pd.read_csv('14000.csv')
df10 = pd.read_csv('16000.csv')
df11 = pd.read_csv('20000.csv')
df12 = pd.read_csv('22000.csv')


# Ghép 2 dataframe lại với nhau theo cột
df_combined = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12], axis=0)

# Xuất kết quả ra file CSV mới
df_combined.to_csv('TinTucGocVNIndex.csv', index=False)

In [ ]:
#*** Thông tin dataset ***
import pandas as pd

# Đọc file CSV
df1 = pd.read_csv('TinTucGocVNIndex.csv')
df1.head()
numrow = df1.shape[0]
print("Số dòng dữ liệu trong dataset là: ", numrow)
#df1.to_csv('TinTuc_Flag.csv', index=False)

In [7]:
#*** Đổi tên cột % Thay đổi thành change_per***
import pandas as pd

# Đường dẫn đến tệp CSV ban đầu và tệp CSV mới
input_file = 'VnIndex01.csv'
output_file = 'VnIndex.csv'

# Đọc tệp CSV vào một DataFrame
df = pd.read_csv(input_file)

# Đổi tên cột
df = df.rename(columns={'% Thay đổi': 'change_per'})

# Ghi lại DataFrame đã thay đổi thành một tệp CSV mới
df.to_csv(output_file, index=False)


In [ ]:
#*** Gán nhãn cho dữ liệu tin tức dựa vào % thay đổi của giá chứng khoán chỉ số VNINDEX****
import pandas as pd

# Đọc dữ liệu từ file csv
df1 = pd.read_csv('TinTucGocVNIndex.csv')
df2 = pd.read_csv('VnIndex.csv')

# Chuyển định dạng của cột 'date' sang định dạng datetime
df1['date'] = pd.to_datetime(df1['date'], format='%Y-%m-%d %H:%M', errors='coerce')
df2['date'] = pd.to_datetime(df2['date'], format='%d/%m/%Y', errors='coerce')

print(df1)
print(df2)
# Hàm chuyển đổi định dạng
def convert_date_format(date):
    return date.strftime("%d/%m/%Y")

# Áp dụng hàm chuyển đổi định dạng cho từng giá trị trong cột
df2['date'] = df2['date'].apply(convert_date_format)
df1['date'] = df1['date'].apply(convert_date_format)

# Tạo cột mới 'change_flag' trong df1
df1['change_flag01'] = ''
            
# Duyệt qua từng dòng trong df1
for i, row in df1.iterrows():
    date = row['date']
    change_per = df2.loc[df2['date'] == date, 'change_per']
    if not change_per.empty:
        if '-' in change_per.iloc[0]:
            df1.at[i, 'change_flag01'] = '0'
        else:
            df1.at[i, 'change_flag01'] = '1'
    else:
        df1.at[i, 'change_flag01'] = '?'
        
#*** Ghi dữ liệu sau gán nhãn vào file csv***
df1.to_csv('TinTuc_Flag.csv', index=True)


In [14]:
#*** Loại bỏ các bản tin không được gán nhãn 0, 1 ***
import pandas as pd
from datetime import datetime, timedelta

# Đọc dữ liệu vào từ file csv
df = pd.read_csv('TinTuc_Flag.csv')
df.dropna(inplace=True)
df1 = df1[df1.iloc[:, -1] != "?"]
df1.to_csv('TinTucAll_Flag.csv', index=False)

In [5]:
#*** Tiền xử lý dữ liệu: Tách từ, loại bỏ từ dừng***
import nltk
import re
import ftfy
import pandas as pd
from nltk.corpus import stopwords
from underthesea import word_tokenize
nltk.download('stopwords')

# Đọc file CSV
df = pd.read_csv('TinTucAll_Flag.csv')

# Chuyển kiểu dữ liệu của cột 'content' từ 'object' sang 'string'
df['content'] = df['content'].astype(str)

# Lưu dataframe vào file CSV
#df.to_csv('news040523str.csv', index=False)
stop_words = set(stopwords.words('vietnamese'))

# define a function to apply the tokenizer and stopword removal to a given text
def preprocess_text(preprocess_text):
    try:
        text1 = ftfy.fix_text(preprocess_text)
        removecharecter= re.sub("[,()':./-]","",text1)
        tokens = word_tokenize(removecharecter)
        filtered_tokens = [token for token in tokens if not token in stop_words]
        return filtered_tokens
    except :
        return 'null'
    # code to handle the exception

# apply the preprocess_text function to the "title" and "summary" columns of your dataset
df['new_content'] = df['content'].apply(preprocess_text)
df.head(20)
#Lưu vào file csv

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hai\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


,date,content,new_content
0,5/10/2023 10:01,"Sau 19 phiÃªn sÃ n liÃªn tiáº¿p, cá»• phiáº¿u ...","[Sau, 19, phiên, sàn, liên tiếp, cổ phiếu, DDG..."
1,5/10/2023 9:04,NghiÃªn cá»©u má»›i nháº¥t cá»§a hai giÃ¡o sÆ°...,"[Nghiên cứu, mới, nhất, hai, giáo sư, Đại học,..."
2,5/10/2023 8:27,"Ã”ng Nguyá»…n Tháº¿ Minh, GiÃ¡m Ä‘á»‘c phÃ¢n t...","[Ông, Nguyễn Thế Minh, Giám đốc, phân tích, Cô..."
3,5/10/2023 8:02,Tin doanh nghiá»‡p DGC -Â CTCP Táº­p Ä‘oÃ n HÃ...,"[Tin, doanh nghiệp, DGC CTCP, Tập đoàn, Hóa ch..."
4,5/10/2023 7:30,"Trong bÃ¡o cÃ¡o má»›i Ä‘Ã¢y, Maybank Investmen...","[Trong, báo cáo, mới đây, Maybank Investment B..."
5,5/10/2023 7:00,Theo sá»‘ liá»‡u tá»« Trung tÃ¢m lÆ°u kÃ½ Chá»...,"[Theo, số liệu, Trung tâm, lưu, ký, Chứng khoá..."
6,5/10/2023 0:10,"Sau quÃ½ Ä‘áº§u nÄƒm mua rÃ²ng khÃ¡ máº¡nh, kh...","[Sau, quý, đầu, năm, mua, ròng, khá, mạnh, khố..."
7,5/10/2023 0:07,CÃ´ng ty Cá»• pháº§n Táº­p Ä‘oÃ n HÃ³a cháº¥t ...,"[Công ty, Cổ phần, Tập đoàn, Hóa chất, Đức Gia..."
8,5/10/2023 0:05,Bá»©c tranh káº¿t quáº£ kinh doanh cá»§a ngÃ n...,"[Bức, tranh, kết quả, kinh doanh, ngành, thép,..."
9,5/10/2023 0:01,TTCK tháº¿ giá»›i Ä‘i vÃ o vÃ¹ng trÅ©ng thÃ´ng...,"[TTCK, thế giới, đi, vùng, trũng, thông tin, T..."


In [6]:
#*** Xóa cột content chứa nội dung chưa xử lý***
# Xóa cột "col_name"
df.drop(columns=['content'], inplace=True)

# Lưu DataFrame vào file CSV mới
df.to_csv('NewsAll_Flag.csv', index=False, index_label=None)

In [15]:
#*** Thông tin dataset ***
import pandas as pd

# Đọc file CSV
df2 = pd.read_csv('TinTucGocVNIdex.csv')
df1 = pd.read_csv('NewsAll_Flag.csv')
numrow1 = df1.shape[0]
numrow2 = df2.shape[0]
print("Số dòng dữ liệu trong data chưa xử lý là: ", numrow2)
print("Số dòng dữ liệu trong dataset sau xử lý là: ", numrow1)

Số dòng dữ liệu trong data chưa xử lý là:  58238
Số dòng dữ liệu trong dataset sau xử lý là:  49780


In [28]:
import pandas as pd
# đọc dữ liệu từ file csv vào dataframe
df = pd.read_csv('NewsAll_Flag.csv')
df.head(300)

,date,new_content,change_flag
0,04/01/2010,"['Thị trường', 'trải', 'một', 'kỳ', 'nghỉ', 'k...",1
1,04/01/2010,"['Hiện tại', 'xử lý tài khoản', 'phụ', 'TTLK',...",1
2,04/01/2010,"['Thị trường', 'chứng khoán', 'Việt Nam', 'tro...",1
3,04/01/2010,"['Đó', 'nhận định', 'chuyên gia', 'chứng khoán...",1
4,04/01/2010,"['Đối với', 'ngành', 'điện', 'Tập đoàn', 'điện...",1
...,...,...,...
295,08/02/2010,"['Theo', 'thông tin', 'ĐTCK', 'tìm hiểu', 'Quỹ...",0
296,08/02/2010,"['Thị trường', 'hồi phục', 'nhẹ', 'cuối', 'phi...",0
297,08/02/2010,"['Sáng kiến', 'phát triển', 'TTTP Châu Á ABMI'...",0
298,08/02/2010,"['Năm', 'Kỷ Sửu', 'có thể', 'coi', 'một', 'năm...",0
